**Startup**

Load Python libraries for scientific computing and librosa MIR library

Set standard parameters for further audio processing

Set paths

In [ ]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-
"""
Created on Sat Nov 18 14:32:47 2017

@author: alexmacbook
"""

import numpy as np
import scipy as sp
#import matplotlib.pyplot as plt, IPython.display as ipd
import librosa
#import librosa.display
import random
import os

CHANNELS = 1
BYTES_PER_SAMPLE = 2
SR = 44100
PAD_DURATION = 0.500

##SOUND STANDARDIZATION
STD_PITCH = 55 #G3
BPM = 100.0 #100bpm

##FOLDERS
PRE_AUDIO = "pre_audio"
POST_SOUNDS = "post_sounds"
POST_SNARE = "post_percussion2"

**Reverb**

simple reverb using Schroedinger algorithm

In [ ]:
def reverb(x, sr):
    # Initialize output array
    output = np.zeros(x.shape)
    
    # Initialize filter coefficients
    a = np.array([0.6, 0.4, 0.2, 0.1, 0.7, 0.6, 0.8])
    R = np.array([700, 900, 600, 400, 450, 390])
    
    # Implement reverb algorithm
    num1 = np.append(0, np.zeros(R[0]-1)); num1 = np.append(num1, 1);
    den1 = np.append(1, np.zeros(R[0]-1)); den1 = np.append(den1, -a[0]);
    d1 = sp.signal.lfilter(num1, den1, x)
    num2 = np.append(0, np.zeros(R[1]-1)); num2 = np.append(num2, 1);
    den2 = np.append(1, np.zeros(R[1]-1)); den2 = np.append(den2, -a[1]);
    d2 = sp.signal.lfilter(num2, den2, x)
    num3 = np.append(0, np.zeros(R[2]-1)); num3 = np.append(num3, 1);
    den3 = np.append(1, np.zeros(R[2]-1)); den3 = np.append(den3, -a[2]);
    d3 = sp.signal.lfilter(num3, den3, x)
    num4 = np.append(0, np.zeros(R[3]-1)); num4 = np.append(num4, 1);
    den4 = np.append(1, np.zeros(R[3]-1)); den4 = np.append(den4, -a[3]);
    d4 = sp.signal.lfilter(num4, den4, x)
    d = d1 + d2 + d3 + d4
    num5 = np.append(a[4], np.zeros(R[4]-1)); num5 = np.append(num5, 1);
    den5 = np.append(1, np.zeros(R[4]-1)); den5 = np.append(den5, a[4]);
    d = sp.signal.lfilter(num5, den5, d)
    num6 = np.append(a[5], np.zeros(R[5]-1)); num6 = np.append(num6, 1);
    den6 = np.append(1, np.zeros(R[5]-1)); den6 = np.append(den6, a[5]);
    d = sp.signal.lfilter(num6, den6, d)
    output = x + a[6]*d;
    # Clip amplitude to minimize distortion
    output *= 0.35
    
    # Ensure correct output array size
    #output.shape=np.original.shape
    
    return output

**Speed Adjustment**

Adjust speed to match 100bpm

In [ ]:
def speed_adjust(x_pre, sr):
    onset_env = librosa.onset.onset_strength(x_pre, sr=sr)
    tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
    x_fast = librosa.effects.time_stretch(x_pre, tempo / BPM)
    return x_fast

**Get pitch**

Get pitch of audio segment by looking at the pitches with magnitudes higher than median

Return pitch as midi note

In [ ]:
def get_pitch(y, sr):
    pitches, magnitudes = librosa.piptrack(y, sr=sr)
    pitches = pitches[magnitudes > np.median(magnitudes)]
    pitches = [int(a) for (a) in pitches]
    pitch = int(librosa.hz_to_midi(sp.stats.mode(pitches)[0]))
    return pitch

**Extract sound**

Simlpy write output to .wav

In [ ]:
def extract_sound(path, song_name):

       
        #pitches, magnitudes = librosa.piptrack(y, sr=sr)
        #pitches = pitches[magnitudes > np.median(magnitudes)]
        #pitches = [int(a) for (a) in pitches]
        #pitch = int(librosa.hz_to_midi(sp.stats.mode(pitches)[0]))
        #pitched_y = librosa.effects.pitch_shift(y, SR, n_steps=STD_PITCH-pitch)
        librosa.output.write_wav(POST_SOUNDS+'/'+song_name, y, sr)
        
        

**Generate Kicks**

Generate kick beat of 8-beat length:

Iterate through the notes and play sample at the respective time

Cut samples at the end of the beat (dirty hack)

In [ ]:
def generate_kicks(path, notes):
    #loading sound file
    y, sr = librosa.load(path, sr=SR)
    
    #defining melody loop boundaries
    beat_length = int(sr*(60/(BPM/8)))
    kick = np.zeros([beat_length])
    sound_length = len(y)
            
    #iterating through the notes
    for n in notes:
        note_wav = np.zeros([beat_length])
        current_sample = int((n/1000) * SR)
        if(current_sample+sound_length <= beat_length):
            note_wav[current_sample:current_sample+sound_length] = y
        else:
            note_wav[current_sample::] = y[:(beat_length - (current_sample+sound_length))]
        kick += note_wav
    return kick

**Generate Snares**

Generate snare beat of 8-beat length:

Iterate through the notes and play sample at the respective time

Cut samples at the end of the beat (dirty hack)

In [ ]:
def generate_snares(path, notes):
    #loading sound file
    y, sr = librosa.load(path, sr=SR)
    
    #defining melody loop boundaries
    beat_length = int(sr*(60/(BPM/8)))
    snare = np.zeros([beat_length])
    sound_length = len(y)
            
    #iterating through the notes
    for n in notes:
        note_wav = np.zeros([beat_length])
        current_sample = int((n/1000) * SR)
        if(current_sample+sound_length <= beat_length):
            note_wav[current_sample:current_sample+sound_length] = y
        else:
            note_wav[current_sample::] = y[:(beat_length - (current_sample+sound_length))]
        snare += note_wav
    return snare

**Generate melody**
Generate melody of 8-beat length:

Iterate through the notes 

    Pitch sounds according to played note (irrespective of octave)

    Play pitches sample at the respective time

    Cut samples at the end of the beat (dirty hack)

In [ ]:
def generate_melody(path, notes):    
    #loading sound file
    y, sr = librosa.load(path, sr=SR)
    
    #defining melody loop boundaries
    beat_length = int(sr*(60/(BPM/8)))
    instrumental = np.zeros([beat_length])
    sound_length = len(y)
    
    #get pitch of sample_sound
    sample_pitch = get_pitch(y, sr)
            
    #iterating through the notes
    for i, n in enumerate(notes):
        pitch = n[0]
        raw_sample_pitch = sample_pitch % 12
        raw_pitch = pitch % 12
        if(pitch >= 60):
            raw_pitch = (pitch % 12)+12

        pitched_y = librosa.effects.pitch_shift(y, sr, n_steps=raw_pitch-raw_sample_pitch)
        note_wav = np.zeros([beat_length])
        current_sample = int((n[1]/1000) * SR)
        if(current_sample+sound_length <= beat_length):
            note_wav[current_sample:current_sample+sound_length] = pitched_y
        else:
            note_wav[current_sample::] = pitched_y[:(beat_length - (current_sample+sound_length))]
        instrumental += note_wav
    return instrumental


**Generate single pitches for live audio**

This function creates the pitched audio for on signature sound exclusively fo the live audio function

Retrieve original pitch of sample

Iterate through supported pitches on online keyboard

    Pitch sample accordingly

    Save pitched sample to .wav

In [ ]:
def generate_single_pitches(path):
    #loading sound file
    y, sr = librosa.load(path, sr=SR)
       
    #get pitch of sample_sound
    sample_pitch = get_pitch(y, sr)
    output_path = path.replace('/post_sounds/', '/post_sounds_pitched/')
    #iterating through the notes
    for i in range(48,66):
        pitch = i
        raw_sample_pitch = sample_pitch % 12
        raw_pitch = pitch % 12
        if(pitch >= 60):
            raw_pitch = (pitch % 12)+12
        pitched_y = librosa.effects.pitch_shift(y, sr, n_steps=raw_pitch-raw_sample_pitch)
        output_filename = output_path + '_' + str(i) + '.wav'
        librosa.output.write_wav(output_filename, pitched_y, SR)

**Generate drumbeat**

Called by flask backend

Generates entire drum loop for given played drum sequence (midi notes + time information)

Saves drum loop (kick + snare) to .wav

In [ ]:
def generate_drumbeat(sound_kick, sound_snare, notes_kick, notes_snare, output_filename):
    kicks = generate_kicks(sound_kick, notes_kick)
    snares = generate_snares(sound_snare, notes_snare)
    audio = kicks + snares
    #audio = audio + (reverbed_audio * 0.3) #Mix reverb
    librosa.output.write_wav(output_filename, audio, SR)

**Generate song from drumbeat**

Called by flask backend

Generates generated melody from played instrumental sequences

Adds some reverb to conceal some of the harsh cuts at the end of some samples

Combines pre-generated drumbeat with melody

Saves the combined beat to .wav

In [ ]:
def generate_song_from_drumbeat(sound_melody, notes_melody, beat_filename, output_filename):
    instrumental = generate_melody(sound_melody, np.array(notes_melody))
    instrumental = instrumental + (reverb(instrumental, SR) * 0.1) #Mix reverb
    drumbeat, sr = librosa.load(beat_filename, sr=SR)
    audio = instrumental + drumbeat
    librosa.output.write_wav(output_filename, audio, SR)

**DEBUG ONLY: Generate song**

Generate entire song with melody, kick and snare sequence (time + sample information)

Add some reverb to conceal some of the harsh cuts at the end of some samples (to drumbeat and instrumental for debug)

Saves the combined beat to .wav

In [ ]:
def generate_song(sound_melody, sound_kick, sound_snare, notes_melody, notes_kick, notes_snare, output_filename):
    instrumental = generate_melody(sound_melody, np.array(notes_melody))
    kicks = generate_kicks(sound_kick, notes_kick)
    snares = generate_snares(sound_snare, notes_snare)
    
    audio =  instrumental + snares + kicks
    reverbed_audio = reverb(audio, SR)
    audio = audio + (reverbed_audio * 0.1) #Mix reverb
    librosa.output.write_wav(output_filename, audio, SR)

**DEBUG ONLY: main**

Call some of the supported functions that will be deployed on the backend with test parameters

In [ ]:
if __name__ == '__main__':
	import os
	for f in os.listdir('/var/www/100bpm/app/sounds/post_sounds'):
		fp = '/var/www/100bpm/app/sounds/post_sounds/' + f
		generate_single_pitches(fp)
"""	generate_song(
		"new_post_sounds/01 Somebody (feat. Jeremih).wav",
		"post_kicks/65 Touchin, Lovin (feat. Nicki Minaj).wav_20.8808708191_25.wav",
		"post_snares/16 Loin (feat. Dany synthé) [Pilule Violette].wav_9.9791841507_9.wav",
		[[55, 920, 201]],
		[50],
		[20],
		"melody.wav"
	)
"""
